단어임베딩: 일반적인 글자를 컴퓨터가 알아들을 수 있는 숫자로 표현하는 방법
ex 원핫이랑 패스트텍스트도 단어임베딩의 일종

In [5]:
import pandas as pd
import requests

In [6]:
res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('ratings_train.txt', 'wb') as f:
    f.write(res.content)
    
nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [7]:
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 전처리

In [11]:
import re #정규 표현식

In [39]:
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [40]:
nsmc.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [41]:
find_hangul(nsmc.loc[0, 'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [42]:
nsmc['document'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
149995    True
149996    True
149997    True
149998    True
149999    True
Name: document, Length: 150000, dtype: bool

In [43]:
# 비어있는 행을 제외한 row행 데이터에서 한글만 추출
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [44]:
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul) #map을 쓰면 뒤의 파라메터로 넘김

In [45]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [46]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [47]:
only_hangul(nsmc.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [48]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [49]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

단어를 실수벡터로 만들어주는 걸 단어임베딩이라고 함
그 방법이 여러가지가 있는데 아까(11/2파일. 모형에 태우니까 단어임베딩을 만들어줌 (8백터, 32백터 그 부분))

In [50]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

## FastText 모형 학습

In [51]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [54]:
#sb = 0(기본값), 1 = (cbow학습), s = 1 :skip-gram학습
# alpha = 0.025 : 학습률
#min_alpha : 학습률을 낮춰주는 역할
#window = 5 : 좌우 몇 개 단어 맥락으로 학습을 할지 결정
#min_count = 5 : 최소 몇 번 나와야 학습을 함
#vector_size = 100(기본값)
model = FastText(vector_size=16) #사이즈가 너무 커서 16으로 줄임

In [55]:
model.build_vocab(corpus_iterable=data)

In [57]:
model.train(corpus_iterable=data, #sentences=data : gensim 4.00|
           epochs=5,
           total_examples=model.corpus_count,
           total_words=model.corpus_total_words
)

(3998626, 5829395)

## 저장

In [58]:
model.save('nsmc.fasttext')

## 임베딩

In [60]:
model = FastText.load('nsmc.fasttext')

In [62]:
model.wv.key_to_index

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [63]:
'히어로' in model.wv.key_to_index #'히어로' in model.wx.vocab : gensim 4.0 미만

True

In [64]:
model.wv['히어로']

array([-0.32572803,  0.78373146,  0.65608186,  0.48279196,  1.1567452 ,
       -0.02693842, -0.92048115, -0.09731456, -0.09753735,  0.12146453,
        0.15098023, -1.1167493 ,  0.6129755 , -0.4848573 , -0.24716902,
        0.38685775], dtype=float32)

In [65]:
len(model.wv['히어로']) #위에 백터 사이즈를 16으로 지정해줬기 때문

16

In [66]:
'슈퍼히어로' in model.wv.key_to_index

False

In [67]:
model.wv['슈퍼히어로'] #인베딩되어있는 데이터를 참고삼아 얘를 가지고 알아서 만들어줌 꼭 글자가 없더라도!! 기존에 있는 걸 합쳐서
#슈퍼히, 퍼히어, 히어로 이런 식으로가 아니라 [ㅅ,ㅠ,ㅍ,ㅓ,ㅎ,ㅣ,ㅇ,ㅓ,ㄹ,ㅗ]이렇게 자모음을 다 쪼개서 비슷한 걸 찾기 때문!

array([-0.17194214,  0.35707274,  0.35715675,  0.19939598,  0.42440614,
        0.06859317, -0.31223527, -0.1144409 ,  0.07932314,  0.02547069,
        0.1428469 , -0.37221673,  0.18542781, -0.18119052, -0.0501512 ,
        0.23472686], dtype=float32)

## 유사도

In [69]:
model.wv.similarity('슈퍼히어로', '히어로') #코사인 유사도로 비교

0.9623246

In [71]:
model.wv.similarity('히어로', '평론가')

0.06682038

In [72]:
model.wv.most_similar('평론가')#평론가랑 유사한 단어들을 쫙 찾아줌 되게 편함! 이 역시 랜덤이라 사람마다 결과가 다르게 나옴

[('평론가들', 0.9767420887947083),
 ('평론', 0.9713326096534729),
 ('평론가들은', 0.954656183719635),
 ('평론가의', 0.9524841904640198),
 ('평론가가', 0.9524068236351013),
 ('전문가', 0.9512952566146851),
 ('점이나', 0.9487007260322571),
 ('점이냐', 0.9466550946235657),
 ('점이야', 0.9448707699775696),
 ('점이죠', 0.938062310218811)]

## FastText를 이용한 감성분석

In [76]:
from gensim.models.fasttext import FastText

ft = FastText.load('nsmc.fasttext')

In [77]:
import pandas as pd

nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [78]:
df = nsmc[nsmc['document'].notnull()]

In [79]:
from sklearn.model_selection import train_test_split

doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], \
                                                       test_size=0.2, random_state=42)

In [80]:
#아까처럼 특수문자빼고 한글만 쓰기위해 처리를 함

import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [81]:
import numpy as np
x_train = np.zeros((1000, 16)) #단어를 1000개로 만들고 각각의 단어는 16벡터. 
# 119996개는 너무 많으니! 문서 하나당 16벡터로 자른다! 원래 단어 하나당 16벡터인데 평균을 내니까 문서 하나가 16벡터다~

In [82]:
doc_train.shape

(119996,)

In [105]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [106]:
doc

'불면의 밤 첫날의 피날레를 멋지게 마무리시켰다.'

1. FestText로 영화댓글 훈련 : 아웃풋 - 단어별 16 vector로 변환 = Embedding vector
<!-- 위에 사이즈16부터 3칸 -->
love = 0.02 0.0004 ....(16실수값)
lion = 0.005 0.007 ....(16실수값)
아 = ....(16실수값)

2. 감성분석 (뭘? 영화댓글)
ex doc1: 아 꿀잼ㅋ 친구랑 봤는데
         2   4       5     7
    doc2 : 개건의 졸작
             6     11 숫자로 바꿔서 원핫인코딩을 통해 신경망에 밀어넣거나 패딩해서 신경망에 밀어넣음(자릿수를 맞춰줘야 하니까)
             
             근데 일케 안 하고
             
     doc1: 아 0.3 0.2 ...(16개)
           꿀잼
           친구랑
     doc2: 개건의
     
     
     

In [102]:
for i, doc in enumerate(doc_train.iloc[:1000]): #열거하다 doc은 문장 하나
    for word in find_hangul(doc):
        print(ft.wv[word])   #list컴프렌션으로 얘를 통으로 만듦
    #list로 나옴

[-1.1030929   4.2045164   1.1303242  -1.8618966   2.741324    1.0344402
 -3.8411777  -3.7643697   3.5804849  -3.948073    0.27003345  3.3723156
 -7.006922    1.3386102   4.4529233   5.33431   ]
[-0.27360025  0.28665334  0.04865921  0.22515973  1.0362111   0.09654833
 -0.47773844 -0.26388073  0.6174784  -0.07138743  0.25028634 -0.08273869
 -0.242489    0.15283047  0.33873937  0.29151863]
[-2.2546756   0.5861678   1.206962   -0.37565017  1.2312362   2.0902328
 -1.6958221   0.5122884   2.0570788   0.05968335  0.24641587 -1.0054601
 -0.6005624   1.1815672   0.7295554   0.46255463]
[-6.039616   -2.3007567   0.64540225  0.3257532   3.2448604   0.7175921
  2.2940345   0.39145517  6.0936227  -0.21357134 -2.1765044  -0.1466059
 -5.3459907  -0.95366186 -0.86739546  2.8505936 ]
[ 1.2804054   0.8797592  -3.1013978   3.3857949   0.21242383  2.106215
  0.75337017 -2.5108254   2.348092    4.0325394   1.0323817  -2.4755416
 -5.5314136  -0.06336643  1.9901322  -0.55127597]
[-0.33214098  0.13737185 -0.0

 -0.7468886   1.6158156   1.1181543  -1.4335877 ]
[ 0.18566437  0.23227346  0.00603385 -0.11379173  4.2841043   1.4216034
 -2.5002325  -0.74804354  3.690884   -0.33034772  3.3138766   2.5382895
 -1.1274157   0.9352566   0.6560923   1.9926627 ]
[-0.50683326  0.62018216  0.09866023  0.22672364 -0.22286603  0.5109594
  0.15341438 -0.7370641   0.23215263 -0.58727795  0.7146792  -0.1650086
 -0.26352707 -0.09526788 -0.6434442   0.61201763]
[-1.7077969   0.36246693  0.15926309  0.9616253  -2.1723516   1.6347458
  2.287186   -4.1319532   0.6692539  -2.2485077   2.6348927  -1.3043469
 -1.8355218   0.79061675 -2.6494324   1.6819049 ]
[ 1.371008    1.0786824  -0.5508337   0.9196765   1.2072253   0.72009444
  0.04413288 -1.1055278   0.8054553   0.79621565 -0.08513308 -0.806884
 -0.1671825  -1.2804948   0.16578268  0.29722735]
[ 0.2533667   1.103655    2.0295858  -0.93847257  1.0480348   1.4149356
  0.31370306 -1.5568292   1.0275203  -0.24362431  2.0458956   0.518754
 -0.43992168  0.5035356  -0.036

 -1.7335637   1.4343396   0.19020016 -4.8247156 ]
[-0.46295217 -0.18338612  1.3099244   1.4345331   1.4092132   0.14169092
 -0.8559621  -0.32402748 -0.30957222 -0.3215286  -1.1722585  -0.25008088
  0.4129746  -0.27918717  0.6046268   1.31452   ]
[-0.9729719   4.7438583   3.607418   -4.699946    0.89158607 -3.2181249
  1.7262063  -1.7039826   2.2699506   0.903756   -6.2957706  -6.007925
 -0.8086597   0.12368093 -3.5725894  -1.557454  ]
[ 1.1566012   2.4419565  -0.16191167 -2.924515    0.60238814 -1.9962718
  0.2155325  -1.5352184   0.8441071  -2.7747047  -0.20454095 -4.1376653
 -0.89370865  0.1486888   1.1431504   0.01843003]
[ 3.284595    0.5631022  -3.5649946   1.9173324   0.01406549  6.1149616
  0.24544653  1.3001442   3.6120248   0.55598736  5.1727943  -0.4166076
  0.71579945 -1.0098609   2.9739552   5.867215  ]
[-0.19592758  0.72328854  0.7784087   0.68144614  0.6386779   0.52606
 -0.3288822  -1.3000268   0.16631474 -0.6901974   1.2142361  -0.3810378
 -0.67262477 -0.09447294 -0.056

  1.8346043  -3.9067664  -1.3690481   1.4803437 ]
[-0.00101974 -0.01309619  0.00373467  0.00504994 -0.00578454  0.00937561
 -0.00963192  0.02882845  0.00893936 -0.00704584  0.01235177 -0.00265473
  0.00394287 -0.02537012  0.00738976  0.01175941]
[ 0.15010831  1.4338596  -0.46065864  0.42167735  0.06215187 -0.43369818
 -0.7080072  -1.2250175  -0.2135864   0.39292794  0.1724515  -2.1963394
  1.3585567  -1.4602364  -0.8084207  -0.06846087]
[-8.976318    1.5072033   1.2301865  -2.0996516  -0.65444314  0.57481885
 -2.4726717   2.5827608   2.3777277  -1.576256    0.4395765  -4.9842367
  0.8241663   0.12038206  7.1917863  -4.3149576 ]
[-4.553258   -1.9286066   0.22414622 -1.4231651   1.0726925   0.35608864
  0.6420225   1.4225734   1.9034677  -1.1656541   1.940187   -2.1022034
 -3.2944329  -0.9464649   0.8798245   0.07019849]
[-0.08667764  0.08595853  0.03935011  0.05762574  0.12172534  0.0304474
 -0.1120261  -0.10280967 -0.04087346 -0.00152978 -0.00894709 -0.12660612
 -0.01899722 -0.05938861

  0.09773101  0.00288624 -0.0622367   0.03420918]
[-0.65543956  0.08717781 -0.31510752  0.73860455  0.5708319   0.0296595
  0.04883226 -0.67147315 -0.10886852  0.2657667   0.33653772 -1.1825365
 -0.29564857 -0.3574126  -0.13214207  0.3633723 ]
[-0.13548611  0.18121734  0.11929201  0.11625465  0.23094139  0.10449108
 -0.04929586 -0.17194894  0.10904729 -0.04336508  0.14152728 -0.20931455
  0.06036948 -0.07685154 -0.03365071  0.16445103]
[-0.56929475  3.8343315   3.2766168  -1.6959314   0.520414   -3.5005836
  2.8662395  -0.9906642   2.7538664  -0.15426444 -3.402558   -4.898982
 -0.50760925  1.9261898  -3.5166526  -1.8997861 ]
[-7.5756917  -2.1710413   2.467089   -4.1682982   1.017314    2.9834685
 -0.666344    1.8829695  -0.67908686  0.68024904  0.05135515 -5.9384336
 -2.8862681  -1.7250915   1.3496245  -2.3282328 ]
[-0.52200764  0.05966098  0.12452681  0.04595576  0.10208421  0.28145272
 -0.23696044 -0.10964219  0.03433582 -0.04248677 -0.06048325 -0.44801953
 -0.15177836  0.05297765  0

...

  0.25897586 -2.2544637   0.9575767  -0.8908588 ]
[-0.27848846  0.60238457  0.8132185   0.15674849  0.5632059   0.44774994
 -0.28690985 -0.5479096   0.13431087 -0.23283844  0.658574   -0.11902733
 -0.26880944 -0.3220214   0.02553695  0.81500036]
[ 0.12470008  2.0446944   1.4703994   0.520181    1.6025038   0.96841997
 -1.5414586  -0.8627129  -1.4373506  -1.2431579   1.0740261   1.1550813
  0.00897614 -2.1779864   0.1334727   1.5448234 ]
[-1.6818957   4.9242644  -1.2449423   2.840505   -1.4355841   3.343152
  2.0642087  -1.5815372  -0.23551065 -4.7551737   4.20635     2.8905385
 -2.2757888  -1.5393366  -4.0094066   4.0418644 ]
[-0.48560876  0.5210258   0.77925     0.16426103  0.3033395   0.06943597
 -0.37845996 -0.4920597   0.35059616 -0.07624173  0.6601157  -1.2511857
  0.2277583  -0.14947486 -0.01674786  0.42334267]
[ 0.22858119  0.30170122  0.29836023  0.10696899  0.21519093  0.05613003
 -0.2066954  -0.01629561 -0.01167923 -0.15010668  0.3013836  -0.2028177
  0.01562936 -0.14339425 -

  0.56286144 -0.5737155  -0.63293034  1.4807148 ]
[-0.95014256  0.9102226   0.16899656 -1.1139609  -0.4017458  -0.04615384
 -0.1169392  -0.7804226  -1.144091   -0.5775851   1.5111305  -3.3038313
  0.6527018  -1.1636894  -0.3382247  -0.86808646]
[-0.20270665  0.13337457 -0.04743876  0.15967853  0.14219186  0.06251968
 -0.15894629 -0.23309053 -0.07056146  0.13278212  0.15263939 -0.32800037
  0.1277399  -0.19000554 -0.04285086 -0.03855768]
[-1.2354578e+00  4.4877997e-01  9.3476579e-04  5.0791246e-01
  2.7152702e-01 -1.4825085e-01  5.3542200e-02 -9.4315571e-01
 -1.7526439e+00  9.2185688e-01  7.9674101e-01 -1.9565142e+00
  9.6302694e-01 -1.1467816e+00 -5.0773706e-02  2.6424890e-02]
[-0.506295    0.33519554 -0.11413896  0.20891286  0.09510597  0.07785009
 -0.36281672 -0.60384417 -0.11687881  0.18696351  0.43789184 -0.93575364
  0.39393732 -0.5626989  -0.44646838 -0.0664386 ]
[ 2.3016498   0.20050995  0.12828206 -0.27180302 -0.01095733  3.9543247
  0.825614   -5.4070945  -3.0938985  -4.365012

 -2.686304   -0.8817608   0.9400405  -1.3107219 ]
[-2.7033608  -0.4099299   0.04821738 -0.03192248 -0.7521989   0.7385439
 -1.646243   -1.7035688  -2.2277343   0.1272216  -0.75846964 -5.414001
  1.4275762  -1.540038    0.39270833 -3.941093  ]
[-0.2819862   0.21556759 -0.06862538  0.31029177 -0.06570377  0.15037595
  0.07396405 -0.5768433  -0.09081668 -0.43023425 -0.1969169  -0.68091434
  0.41758844 -0.25540286  0.04760102  0.03771975]
[-0.6837629  -0.14894173 -0.4834558   0.6588745   0.724743    0.11884354
  0.09105004 -0.615579    0.05705402  0.31315973  0.3147444  -1.2501162
 -0.7624069  -0.15072593  0.02357496  0.32239023]
[ 1.7987429   1.8255322  -1.8236148   1.1510115   3.4557104  -0.5654066
 -0.20586425 -1.0867956   4.468727    1.447707    0.45177045  0.04427642
 -0.5852789   0.6192996   0.82581323  0.61364496]
[-0.8698814  -0.00540247 -0.356301    0.9159162   0.01604698 -0.24349986
  0.35492292 -0.2848562   0.09687522  0.32942012 -0.03750245 -0.96064675
 -0.15981942 -0.00827677 

  0.1333231  -0.8507611  -0.00990913 -0.7679664 ]
[-0.9729719   4.7438583   3.607418   -4.699946    0.89158607 -3.2181249
  1.7262063  -1.7039826   2.2699506   0.903756   -6.2957706  -6.007925
 -0.8086597   0.12368093 -3.5725894  -1.557454  ]
[-0.25626862 -0.31512868 -0.06135405  0.2683562   1.966753   -0.12807132
  0.13896628 -0.09373214  0.36655512 -0.05806104  0.19183849 -0.81762123
 -0.6664346   0.13480501  0.73658526  0.07431746]
[-1.4946947   0.45899007 -1.5945495   0.9143316  -1.6298845   0.3581414
 -1.3722202  -1.0199621  -1.9567449   1.1663135   0.9352532  -4.370553
  2.174616   -3.7735274  -1.5495243  -1.9362581 ]
[-1.1719952   0.4859943   0.06926525  0.08491904  0.02737626  0.36666074
 -0.7191236  -0.72654325 -0.17880383 -0.08764112  0.76009357 -1.5963573
  0.62157714 -0.8756191  -0.27691203 -0.26195508]
[-5.7276025  -0.21342048  3.4901376  -1.015203   -2.2579098  -2.4707124
 -2.4414017   0.65152305  4.1543617  -2.7999709   1.1559476  -3.8917742
 -0.2920298   2.0167832   4.5

  1.5826006  -0.09741919 -0.45287725  4.0493975 ]
[-0.9729719   4.7438583   3.607418   -4.699946    0.89158607 -3.2181249
  1.7262063  -1.7039826   2.2699506   0.903756   -6.2957706  -6.007925
 -0.8086597   0.12368093 -3.5725894  -1.557454  ]
[ 1.9347659   1.8809612  -0.48896807  0.41726527  0.77978516  1.1232636
  0.2676843  -1.2933704   0.26325533  0.77546084 -0.2684215  -1.7371217
 -0.09089691 -2.0131824   0.22311686  0.05889464]
[ 0.00500579  0.0128     -1.0598463   0.8166541   2.3490155  -0.6068074
 -0.9002248  -1.18375     0.77214247  0.6897028   0.4378805  -1.0403118
 -1.0576246  -0.53940594  0.5901845   0.08847331]
[-1.5202522  -0.06624755 -1.3069496  -0.63248336  2.0109282   0.771208
  0.48941448 -0.87383705  0.90187556  1.8931197   0.56273705 -3.9464056
 -0.6229775   2.1311707   0.9376596  -2.307776  ]
[ 0.01278723  0.7635193   0.05267526  0.5894348   1.3137447   0.488494
 -0.7884036  -0.42031953  1.0502837  -0.27587587  0.20803437 -0.13088793
 -0.87085605 -0.17768869  0.5727

In [98]:
for i, doc in enumerate(doc_train.iloc[:1000]): #doc은 문장 하나
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv] 
#     [ft.wv[word] : 워드단위로 뽑아내서 인베딩백터 16백터로 뽑아내겠다.
#     if vs and i == 1:
    if vs:
#         print(doc) #첫번째 댓글에 대해 인베딩백터를 찍으려고 함
#         print(len(vs)) #그 단어는 6개(띄어쓰기로 구분) 총 벡터는 6* 16개
#         print(vs[0]) #단어는 하나당 16백터로 존재할 것. (16벡터로 맨 위에 훈련시켰으니)
        x_train[i,]=(np.mean(vs, axis=0)) #세로 1000개의 문서의 가로는 16. 여러 doc의 평균이 16으로 나옴
    else:
        print(doc)
        
# vs #단어임베딩 백터가 나옴. 한 리스트가 하나의 문장

once upon a dream
good
good
it's jjangge movie
Good message
